In [53]:
import pandas as pd

df = pd.read_csv('raw/삼성전자.csv')

In [55]:
import csv

row_len = 30 # each row has 30 items, the last item being the buy/sell signal
iter_len = len(df['Close'])-row_len
start_q = list(df['Close'][:row_len])

filename = 'prep1/삼성전자.csv'

# Labeling: buy = 1, sell = 0

with open(filename, 'w', newline='') as f:
    wt = csv.writer(f)
    wt.writerow(['Day '+str(i) for i in range(1,row_len)]+['Buy or Sell'])
    for i in range(iter_len):
        temp = start_q.copy()
        if temp[-1]>temp[-2]:
            label = 1
        else: label = 0
        m = max(temp[:-1])
        for idx, el in enumerate(temp):
            temp[idx]=el/m
        temp[-1]=label
            
        wt.writerow(temp)
        del start_q[0]
        start_q.append(df['Close'][i+row_len])
        

In [56]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


f = pd.read_csv(filename)
labels = f.axes[1]
rawX = f.drop(columns=labels[-1], axis=1).values
rawY = f.drop(columns=labels[:-1], axis=1).values

In [57]:
newY = []
empty = np.zeros((2,), dtype=int)

for val in rawY:
    row = empty.copy()
    row[val]=100
    newY.append(row)

x=np.array(rawX)
y=np.array(newY)

if torch.cuda.is_available():
    x=torch.tensor(x).cuda()
    y=torch.tensor(y).cuda()
    print("CUDA available")
else:
    x=torch.tensor(x)
    y=torch.tensor(y)
    

In [58]:
class Net(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.l1 = nn.Linear(D_in, H)
        self.h1 = nn.Linear(H, 2*H)
        self.h2 = nn.Linear(2*H, 4*H)
        self.l2 = nn.Linear(4*H, D_out)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        y = F.relu(self.l2(x))
        return y


D_in, D_out = 29, 2 #input, output 

def run(H, lr, epochs):
  
    #training
    
    model = Net(D_in, H, D_out)
    if torch.cuda.is_available(): model = model.cuda()
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for t in range(epochs):
        y_pred = model(x.float())
        loss = criterion(y_pred, y.float())
        print(t, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  ###############################################################################
  
    #evaluation

    correct = 0
    total = 0

    preds = {}
    truths = {}

    for idx, inp in enumerate(x):
        total+=1
        y_pred = model(inp.float()).cpu().detach().numpy()
        y_truth = list(newY[idx]).index(max(newY[idx]))
        y_pred = list(y_pred).index(max(y_pred))  

        if y_truth == y_pred: correct+=1
        if y_truth in truths: truths[y_truth]+=1
        else: truths[y_truth]=1
        if y_pred in preds: preds[y_pred]+=1
        else: preds[y_pred]=1

    #print(f'{correct*100/total}% correct\n')
  
    return round(correct*100/total,1)



In [61]:
accs = []
accs.append(run(48, 1e-5, 1000))
print(accs[0])

0 54477516.0
1 42731580.0
2 3155812864.0
3 6165971337216.0
4 1.58914267073472e+28
5 54480000.0
6 54480000.0
7 54480000.0
8 54480000.0
9 54480000.0
10 54480000.0
11 54480000.0
12 54480000.0
13 54480000.0
14 54480000.0
15 54480000.0
16 54480000.0
17 54480000.0
18 54480000.0
19 54480000.0
20 54480000.0
21 54480000.0
22 54480000.0
23 54480000.0
24 54480000.0
25 54480000.0
26 54480000.0
27 54480000.0
28 54480000.0
29 54480000.0
30 54480000.0
31 54480000.0
32 54480000.0
33 54480000.0
34 54480000.0
35 54480000.0
36 54480000.0
37 54480000.0
38 54480000.0
39 54480000.0
40 54480000.0
41 54480000.0
42 54480000.0
43 54480000.0
44 54480000.0
45 54480000.0
46 54480000.0
47 54480000.0
48 54480000.0
49 54480000.0
50 54480000.0
51 54480000.0
52 54480000.0
53 54480000.0
54 54480000.0
55 54480000.0
56 54480000.0
57 54480000.0
58 54480000.0
59 54480000.0
60 54480000.0
61 54480000.0
62 54480000.0
63 54480000.0
64 54480000.0
65 54480000.0
66 54480000.0
67 54480000.0
68 54480000.0
69 54480000.0
70 54480000.0

553 54480000.0
554 54480000.0
555 54480000.0
556 54480000.0
557 54480000.0
558 54480000.0
559 54480000.0
560 54480000.0
561 54480000.0
562 54480000.0
563 54480000.0
564 54480000.0
565 54480000.0
566 54480000.0
567 54480000.0
568 54480000.0
569 54480000.0
570 54480000.0
571 54480000.0
572 54480000.0
573 54480000.0
574 54480000.0
575 54480000.0
576 54480000.0
577 54480000.0
578 54480000.0
579 54480000.0
580 54480000.0
581 54480000.0
582 54480000.0
583 54480000.0
584 54480000.0
585 54480000.0
586 54480000.0
587 54480000.0
588 54480000.0
589 54480000.0
590 54480000.0
591 54480000.0
592 54480000.0
593 54480000.0
594 54480000.0
595 54480000.0
596 54480000.0
597 54480000.0
598 54480000.0
599 54480000.0
600 54480000.0
601 54480000.0
602 54480000.0
603 54480000.0
604 54480000.0
605 54480000.0
606 54480000.0
607 54480000.0
608 54480000.0
609 54480000.0
610 54480000.0
611 54480000.0
612 54480000.0
613 54480000.0
614 54480000.0
615 54480000.0
616 54480000.0
617 54480000.0
618 54480000.0
619 544800